# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [12]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!

In [13]:
# Load environment variables in a file called .env
load_dotenv()

# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

class LLM_Model:
    api_key: str
    model: str
    base_url: str
    openai: OpenAI

    def __init__(self, key, model, url):
        self.api_key_name = key
        self.model = model
        self.base_url = url
        self.api_key = os.getenv(key)
        if not self.api_key:
            raise ValueError ("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
        self.openai = OpenAI(base_url=url, api_key=self.api_key)

    def getResponse(self, messages, streaming=False, tools=None):
        response = self.openai.chat.completions.create(model=self.model, messages=messages, stream=streaming, tools=tools)
        if streaming == False:
            return response.choices[0].message.content
        else:
            result = ""
            for chunk in response:
                result += chunk.choices[0].delta.content or ""
                yield result
        
deepseek = LLM_Model("DEEPSEEK_API_KEY", "deepseek-chat", "https://api.deepseek.com")
deepseekR1 = LLM_Model("DEEPSEEK_API_KEY", "deepseek-reasoner", "https://api.deepseek.com")
llama = LLM_Model("OPENAI_API_KEY", "llama3.2", "http://localhost:11434/v1")
gpt = LLM_Model("OPENAI_API_KEY", "gpt-4o", "https://api.openai.com/v1")

llms = {"DeepSeek": deepseek, "DeepSeek R1": deepseekR1, "Llama": llama, "ChatGpt": gpt}

# A generic system message - no more snarky adversarial AIs!
system_message = "You are a helpful assistant"

In [14]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_openai(prompt, model):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    return model.getResponse(messages, True)


In [15]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [16]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_openai(prompt, gpt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="DeepSeekV3":
        result = stream_openai(prompt, deepseek)
    elif model=="DeepSeekR1":
        result = stream_openai(prompt, deepseekR1)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:", lines=6), gr.Dropdown(["GPT", "Claude", "DeepSeekV3", "DeepSeekR1"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=True, )